Set Shots - Expected Goal Model - Model Tuning and Building - GBM

In [30]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/")

In [31]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

from expected_score_model.config import set_shots_file_path
# from expected_score_model.domain.preprocessing.data_preprocessor import DataPreprocessor
# from expected_score_model.domain.contracts.mappings import Mappings
from expected_score_model.domain.contracts.modelling_data_contract import ModellingDataContract
from expected_score_model.domain.modelling.hyperparameter_tuning import XGBHyperparameterTuner
from expected_score_model.domain.modelling.supermodel import SuperXGBClassifier
from expected_score_model.domain.modelling.optuna_xgb_param_grid import OptunaXGBParamGrid

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Model Version

In [89]:
model_version = 1
model_name = 'expected_goal_set'
model_file_name = model_name + '_v' + str(model_version)

model_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models"
prediction_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/predictions/"

preprocessor_file_name = 'preprocessor_v' + str(model_version)
preprocessor_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/preprocessors/"

In [90]:
RESPONSE = ModellingDataContract.RESPONSE_GOAL

Load Data

In [91]:
df_modelling = pd.read_csv(set_shots_file_path)
df_modelling.tail()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Goal,Behind,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,random100,ModellingFilter
11052,208,possGain,behind,1663.0,4,634,683.0,Geelong,Geelong,Tom Hawkins,Tom_Hawkins,Kick,41.0,25.0,ineffective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,0,1,1,1,0,106.0,33.0,73.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,41.0,48.0,48.0,26.0,25.0,21.0,21.0,14.0,683.0,644.0,642.0,641.0,49.0,39.0,8.062258,39.0,25.0,46.324939,0.570040,32.660913,0.116867,6.695990,0.364585,20.889203,1521.0,3.663562,True,10,True
11053,212,kickIn,goal,1730.0,4,787,885.0,Geelong,Geelong,Patrick Dangerfield,Patrick_Dangerfield,Kick,51.0,43.0,ineffective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,0,1,1,1,0,108.0,34.0,74.0,Kick,Contested Mark,Contest Target,Kick Inside 50 Result,51.0,53.0,53.0,53.0,43.0,38.0,38.0,38.0,885.0,852.0,851.0,850.0,98.0,33.0,5.385165,29.0,43.0,51.865210,0.977446,56.003541,0.069260,3.968302,0.214332,12.280314,841.0,3.367296,True,66,True
11054,213,centreBounce,goal,1754.0,4,956,979.0,Sydney,Sydney,Paddy McCartin,Paddy_McCartin,Kick,66.0,0.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,6,0,6,115.0,34.0,81.0,Kick,Free For,Kick Inside 50 Result,Kick Into F50,66.0,74.0,74.0,27.0,0.0,0.0,0.0,17.0,979.0,977.0,968.0,967.0,23.0,2.0,8.000000,14.0,0.0,14.000000,0.000000,0.000000,0.482343,27.636207,2.588598,148.315731,196.0,2.639057,True,12,True
11055,219,centreBounce,goal,1846.0,4,1252,1266.0,Sydney,Sydney,Chad Warner,Chad_Warner,Kick,26.0,3.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,6,0,6,121.0,40.0,81.0,Kick,Uncontested Mark,Ground Kick,Handball,26.0,18.0,-5.0,-2.0,3.0,2.0,-4.0,3.0,1266.0,1265.0,1264.0,1260.0,14.0,1.0,8.062258,54.0,3.0,54.083269,0.055499,3.179830,0.118569,6.793500,0.365993,20.969862,2916.0,3.988984,True,18,True
11056,228,centreBounce,goal,1912.0,4,1519,1560.0,Geelong,Geelong,Jeremy Cameron,Jeremy_Cameron,Kick,41.0,21.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,6,6,0,127.0,46.0,81.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,41.0,49.0,49.0,25.0,21.0,17.0,17.0,33.0,1560.0,1535.0,1532.0,1531.0,41.0,25.0,8.944272,39.0,21.0,44.294469,0.493941,28.300756,0.127885,7.327246,0.400462,22.944787,1521.0,3.663562,True,93,False


In [92]:
training_data = df_modelling[df_modelling[ModellingDataContract.TRAIN_TEST_SPLIT_COL]]
test_data = df_modelling[~df_modelling[ModellingDataContract.TRAIN_TEST_SPLIT_COL]]

In [93]:
X_train, y_train = training_data.drop(columns=[RESPONSE]), training_data[RESPONSE]
X_test, y_test = test_data.drop(columns=[RESPONSE]), test_data[RESPONSE]

In [94]:
X_test.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Behind,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,random100,ModellingFilter
6,72,possGain,goal,696.0,2,462,506.0,Sydney,Sydney,Isaac Heeney,Isaac_Heeney,Kick,47.0,35.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,6,0,6,20.0,28.0,-8.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,47.0,53.0,53.0,42.0,35.0,27.0,27.0,41.0,506.0,468.0,466.0,465.0,44.0,38.0,10.000000,31.0,35.0,46.754679,0.845930,48.468229,0.091187,5.224603,0.284262,16.287036,961.0,3.433987,True,90,False
10,108,possGain,goal,982.0,2,1782,1859.0,Sydney,Sydney,Sam Reid,Sam_Reid,Kick,40.0,36.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,6,0,6,50.0,46.0,4.0,Kick,Uncontested Mark,Kick,Uncontested Mark,40.0,44.0,43.0,50.0,36.0,32.0,48.0,40.0,1859.0,1835.0,1833.0,1814.0,77.0,24.0,5.656854,38.0,36.0,52.345009,0.758378,43.451842,0.089092,5.104600,0.275545,15.787586,1444.0,3.637586,True,95,False
27,15,ballUp,behind,140.0,1,453,464.0,Collingwood,Collingwood,Brody Mihocek,Brody_Mihocek,Kick,31.0,-23.0,ineffective,True,NaN,141,160,Collingwood,Western Bulldogs,right,202101_Collingwood_WesternBulldogs,202101,2021,1,1,1,0,0.0,18.0,-18.0,Kick,Mark On Lead,Kick,Bounce,31.0,34.0,-7.0,-8.0,-23.0,-32.0,-5.0,-5.0,464.0,461.0,460.0,460.0,11.0,3.0,9.486833,49.0,23.0,54.129474,0.438859,25.144786,0.107406,6.153916,0.331520,18.994682,2401.0,3.891820,True,96,False
39,156,throwIn,miss,1357.0,3,766,816.0,Western Bulldogs,Western Bulldogs,Josh Bruce,Josh_Bruce,Kick,42.0,38.0,clanger,True,NaN,141,160,Collingwood,Western Bulldogs,right,202101_Collingwood_WesternBulldogs,202101,2021,0,0,0,0,36.0,48.0,-12.0,Kick,Free For,Kick Inside 50 Result,Kick Into F50,42.0,46.0,46.0,-1.0,38.0,34.0,34.0,61.0,816.0,784.0,772.0,771.0,50.0,32.0,5.656854,38.0,38.0,53.740115,0.785398,45.000000,0.084510,4.842076,0.260959,14.951857,1444.0,3.637586,True,91,False
41,195,possGain,miss,1617.0,3,1666,1697.0,Western Bulldogs,Western Bulldogs,Aaron Naughton,Aaron_Naughton,Kick,46.0,-44.0,ineffective,True,NaN,141,160,Collingwood,Western Bulldogs,right,202101_Collingwood_WesternBulldogs,202101,2021,1,1,0,1,36.0,62.0,-26.0,Kick,Mark On Lead,Kick,Handball Received,46.0,46.0,14.0,10.0,-44.0,-42.0,-32.0,-28.0,1697.0,1670.0,1668.0,1667.0,31.0,27.0,2.000000,34.0,44.0,55.605755,0.912908,52.305760,0.070609,4.045598,0.217612,12.468227,1156.0,3.526361,True,81,False


Preprocess Data

In [95]:
# preprocessor = DataPreprocessor()
# preprocessor.fit(X_train)

# X_train_preproc = preprocessor.transform(X_train)
# X_test_preproc = preprocessor.transform(X_test)

In [96]:
X_train_preproc = X_train[ModellingDataContract.modelling_feature_list]
X_test_preproc = X_test[ModellingDataContract.modelling_feature_list]

In [97]:
X_train_preproc.shape, X_test_preproc.shape

((8729, 2), (2328, 2))

In [98]:
X_train_preproc.head()

,Distance_to_Middle_Goal,Angle_to_Middle_Goal
0,52.009614,0.908067
1,43.600459,0.931882
2,36.674242,0.451453
3,33.015148,0.957589
4,46.572524,0.260602


In [99]:
X_test_preproc.head()

,Distance_to_Middle_Goal,Angle_to_Middle_Goal
6,46.754679,0.845930
10,52.345009,0.758378
27,54.129474,0.438859
39,53.740115,0.785398
41,55.605755,0.912908


Optuna Hyperparameter Tuning Class - HyperParameterTuner & XGBHyperparameterTuner

In [74]:
xgb_tuner = XGBHyperparameterTuner(X_train_preproc, y_train, monotonicity_constraints = {})

In [75]:
xgb_tuner.training_data.head()

,Distance_to_Middle_Goal,Angle_to_Middle_Goal
0,52.009614,0.908067
1,43.600459,0.931882
2,36.674242,0.451453
3,33.015148,0.957589
4,46.572524,0.260602


In [76]:
xgb_tuner.tune_hyperparameters()

[I 2023-06-28 14:02:50,715] A new study created in memory with name: no-name-b9a0acdd-04bd-49cb-a2e7-d0b0c2939ca0
[I 2023-06-28 14:02:50,779] Trial 0 finished with value: 0.6828605221155181 and parameters: {'max_depth': 15, 'min_child_weight': 18, 'eta': 0.013386383512337719, 'gamma': 0.0009973010014321696, 'lambda': 0.0010271708771546846, 'alpha': 0.3002441719735701, 'subsample': 0.23141985852845717, 'colsample_bytree': 0.7967579332110315}. Best is trial 0 with value: 0.6828605221155181.
[I 2023-06-28 14:02:50,814] Trial 1 finished with value: 0.6693839423823175 and parameters: {'max_depth': 15, 'min_child_weight': 26, 'eta': 0.034845244393341614, 'gamma': 0.003557989151361737, 'lambda': 0.000551408532174095, 'alpha': 0.8494958783696482, 'subsample': 0.6856829521855925, 'colsample_bytree': 0.688663027503217}. Best is trial 1 with value: 0.6693839423823175.
[I 2023-06-28 14:02:50,842] Trial 2 finished with value: 0.6750894914804696 and parameters: {'max_depth': 16, 'min_child_weight': 

Number of finished trials:  100
Best trial:
  Value: 0.5791826230049133
  Params: 
    max_depth: 2
    min_child_weight: 66
    eta: 0.6132005591427557
    gamma: 0.2245672988440354
    lambda: 0.001434080210414414
    alpha: 7.353533211094619
    subsample: 0.8193193033251163
    colsample_bytree: 0.8302789610902116


In [77]:
params = xgb_tuner.get_best_params()
params

{'max_depth': 2,
 'min_child_weight': 66,
 'eta': 0.6132005591427557,
 'gamma': 0.2245672988440354,
 'lambda': 0.001434080210414414,
 'alpha': 7.353533211094619,
 'subsample': 0.8193193033251163,
 'colsample_bytree': 0.8302789610902116}

Training Model - SuperXGBClassifier class for training and predictions

In [78]:
params['objective'] = OptunaXGBParamGrid.error
params['num_rounds'] = 1000
params['early_stopping_rounds'] = 50
params['verbosity'] = 1
params['monotone_constraints'] = {}

In [79]:
super_xgb = SuperXGBClassifier(X_train = X_train_preproc, 
                               y_train = y_train, 
                               X_test = X_test_preproc, 
                               y_test = y_test, 
                               params = params)

In [80]:
super_xgb.fit()

[0]	validation_0-logloss:0.66833	validation_1-logloss:0.67295
[1]	validation_0-logloss:0.66264	validation_1-logloss:0.66857
[2]	validation_0-logloss:0.61449	validation_1-logloss:0.62363
[3]	validation_0-logloss:0.61276	validation_1-logloss:0.62270
[4]	validation_0-logloss:0.60022	validation_1-logloss:0.61120
[5]	validation_0-logloss:0.59908	validation_1-logloss:0.61032
[6]	validation_0-logloss:0.59567	validation_1-logloss:0.60811
[7]	validation_0-logloss:0.59503	validation_1-logloss:0.60759
[8]	validation_0-logloss:0.59387	validation_1-logloss:0.60699
[9]	validation_0-logloss:0.59360	validation_1-logloss:0.60726
[10]	validation_0-logloss:0.59300	validation_1-logloss:0.60716
[11]	validation_0-logloss:0.59247	validation_1-logloss:0.60722
[12]	validation_0-logloss:0.59200	validation_1-logloss:0.60755
[13]	validation_0-logloss:0.59182	validation_1-logloss:0.60755
[14]	validation_0-logloss:0.59157	validation_1-logloss:0.60803
[15]	validation_0-logloss:0.59143	validation_1-logloss:0.60790
[1

In [81]:
super_xgb.xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8302789610902116, early_stopping_rounds=50,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.2245672988440354, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.6132005591427557, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=66, missing=nan, monotone_constraints={},
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [82]:
super_xgb.xgb_model.get_booster().feature_names

['Distance_to_Middle_Goal', 'Angle_to_Middle_Goal']

In [83]:
train_preds = super_xgb.predict(X_train_preproc)
test_preds = super_xgb.predict(X_test_preproc)

In [84]:
train_probas = super_xgb.predict_proba(X_train_preproc)[:, 1]
test_probas = super_xgb.predict_proba(X_test_preproc)[:, 1]

Check Average Predictions

In [85]:
train_preds.mean(), training_data[RESPONSE].mean(), test_preds.mean(), test_data[RESPONSE].mean()

(0.5320197044334976,
 0.5351128422499714,
 0.5322164948453608,
 0.5236254295532646)

Export model

In [86]:
super_xgb.export_model(model_output_path + "/" + model_file_name + ".joblib")

Export data and predictions

In [87]:
train_info = training_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
train_export = pd.concat([train_info, X_train_preproc.reset_index(drop=True)], axis=1)
train_export['xgb_preds'] = train_preds
train_export['xgb_probas'] = train_probas
train_export.to_csv(prediction_output_path + 'train_predictions_' + model_file_name + '.csv', index = False)
train_export.head()

,Match_ID,Chain_Number,Order,Goal,Distance_to_Middle_Goal,Angle_to_Middle_Goal,xgb_preds,xgb_probas
0,202101_BrisbaneLions_Sydney,17,183.0,0,52.009614,0.908067,0,0.200988
1,202101_BrisbaneLions_Sydney,28,295.0,0,43.600459,0.931882,0,0.304358
2,202101_BrisbaneLions_Sydney,34,354.0,0,36.674242,0.451453,1,0.600964
3,202101_BrisbaneLions_Sydney,53,537.0,0,33.015148,0.957589,0,0.486338
4,202101_BrisbaneLions_Sydney,59,578.0,1,46.572524,0.260602,1,0.548933


In [88]:
test_info = test_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
test_export = pd.concat([test_info, X_test_preproc.reset_index(drop=True)], axis=1)
test_export['xgb_preds'] = test_preds
test_export['xgb_probas'] = test_probas
test_export.to_csv(prediction_output_path + 'test_predictions_' + model_file_name + '.csv', index = False)
test_export.head()

,Match_ID,Chain_Number,Order,Goal,Distance_to_Middle_Goal,Angle_to_Middle_Goal,xgb_preds,xgb_probas
0,202101_BrisbaneLions_Sydney,72,696.0,1,46.754679,0.845930,0,0.310845
1,202101_BrisbaneLions_Sydney,108,982.0,1,52.345009,0.758378,0,0.244665
2,202101_Collingwood_WesternBulldogs,15,140.0,0,54.129474,0.438859,0,0.271621
3,202101_Collingwood_WesternBulldogs,156,1357.0,0,53.740115,0.785398,0,0.201273
4,202101_Collingwood_WesternBulldogs,195,1617.0,0,55.605755,0.912908,0,0.163664


Save preprocessor

In [ ]:
# joblib.dump(preprocessor, preprocessor_output_path + preprocessor_file_name + ".joblib")